In [1]:
import os
from collections import defaultdict
import time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.datasets import dump_svmlight_file
import scipy
import xgboost as xgb

from utils.utils import *


for dirname, _, filenames in os.walk('../data/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



In [12]:
# Load in training and test data. Update training data with updated version

train = pd.read_csv("../data/train.csv", index_col="seq_id")
train_updates = pd.read_csv("../data/train_updates_20220929.csv", index_col="seq_id") # Updated version

to_drop = train_updates[train_updates["tm"].isnull()].index
train.drop(index=to_drop, inplace=True)
train_updates.drop(index=to_drop, inplace=True)

to_change = train_updates.index
train.drop(index=to_change, inplace=True)
train = pd.concat([train, train_updates])

train.drop(train[train.pH.isnull()].index, inplace=True)

test = pd.read_csv("../data/test.csv", index_col="seq_id")


pad_length = 4
shingle_size = pad_length + 1

train.insert(2, "length", train.protein_sequence.str.len())
test.insert(2, "length", test.protein_sequence.str.len())

cutoff_length = int(train.length.quantile(0.95))

train.insert(1, "exp_tm", np.e ** (-1/train.tm))

train.drop(train[train.length > cutoff_length].index, inplace=True)

train.insert(1, "padded_sequence", 
             [string_pad(sequence, cutoff_length + 2*pad_length, pad_length) 
              for sequence in train.protein_sequence])
test.insert(1, "padded_sequence", 
             [string_pad(sequence, cutoff_length + 2*pad_length, pad_length) 
              for sequence in test.protein_sequence])

amino_acid_list = list(sorted({aa for sequence in test.protein_sequence for aa in set(sequence) }))


train_split = 0.75

column_labels = train.drop(columns=["tm", "protein_sequence", "data_source", "exp_tm"]).columns
X = train.drop(columns=["tm", "protein_sequence", "data_source", "exp_tm"]).values
Y = train["exp_tm"].values

X_train, Y_train, X_test, Y_test = test_train_split(X, Y, train_split)
print(column_labels)


print(X.shape)


amino_acid_data = pd.read_csv("../data/aminoacids.csv")
amino_acid_data.drop(columns=["Name", "Abbr", "Molecular Formula", "Residue Formula"], inplace=True)
amino_acid_data = amino_acid_data[amino_acid_data.Letter.isin(amino_acid_list)]

buffer_residue = {
    "Letter": "X",
    "Molecular Weight": 0,
    "Residue Weight": 0,
    "pKa1" : np.NaN,
    "pKb2" : np.NaN,
    "pKx3" : np.NaN,
    "pl4"  : np.NaN,
    "H" : 0,
    "VSC" : 0,
    "P1" : 0, 
    "P2" : 0,
    "SASA" : 0,
    "NCISC" : 0,
    "carbon" : 0,
    "hydrogen" : 0,
    "nitrogen" : 0,
    "oxygen" : 0,
    "sulfur" : 0,
}

amino_acid_data = amino_acid_data.reset_index().drop(columns="index")
amino_acid_data.loc[len(amino_acid_data.index)] = list(buffer_residue.values())

print(amino_acid_data.columns)

amino_acid_data_dictionary = amino_acid_data.set_index('Letter').T.to_dict('list')
amino_acid_data_dictionary = {key: np.array(value, dtype=np.float32) for key, value in amino_acid_data_dictionary.items()}


column_names = [f"{col}_{letter_num}" 
                for letter_num in range(shingle_size) 
                for col in amino_acid_data.drop(columns="Letter").columns]

shingled_dimensions = ((cutoff_length + 2*pad_length - shingle_size) * X_train.shape[0],
                       (amino_acid_data.columns.shape[0]-1) * shingle_size)
X_train_shingled = np.zeros(shingled_dimensions, dtype=np.float32)
Y_train_shingled = np.zeros(shingled_dimensions[0], dtype=np.float32)

np.set_printoptions(suppress=True)
print((cutoff_length + 2*pad_length - shingle_size))

i = 0
t0 = time.time()
for x, y in zip(X_train, Y_train):
    sequence = x[0]
    pH = x[1]
    length = x[2]
    
    row = np.array([amino_acid_data_dictionary[amino_acid] for amino_acid in sequence])
    row[:, [2,3, 4,5]] = row[:, [2,3, 4,5]] - pH
    row[:, 3] = row[:, 3] - (14-pH)
    row[:, 2:6] = 10 ** (row[:, 2:6])
    row[:, 2:6] = 1 / (row[:, 2:6] + 1)
    row = np.nan_to_num(row, nan=1.0)
    row = np.array([row[j:j+shingle_size].reshape(-1) for j in range(row.shape[0]-shingle_size)])
    
    X_train_shingled[i:i+(cutoff_length + 2*pad_length - shingle_size)] = row
    Y_train_shingled[i:i+(cutoff_length + 2*pad_length - shingle_size)] = y

    i += (cutoff_length + 2*pad_length - shingle_size)

    print(f"At {int(100 * i/shingled_dimensions[0])}%. Runtime: {int((time.time()-t0)/60)} minutes", end="\r")
print()

dump_svmlight_file(X_train_shingled, Y_train_shingled, "train_values.svm.txt")
#X_train_shingled = pd.DataFrame(X_train_shingled, columns=column_names)
#Y_train_shingled = pd.DataFrame(Y_train_shingled.T, columns=["Tm"]).Tm
#X_train_shingled.to_csv("X_train.csv")
#Y_train_shingled.to_csv("Y_train.csv")


shingled_dimensions = ((cutoff_length + 2*pad_length - shingle_size) * X_test.shape[0],
                       (amino_acid_data.columns.shape[0]-1) * shingle_size)
X_test_shingled = np.zeros(shingled_dimensions, dtype=np.float32)
Y_test_shingled = np.zeros(shingled_dimensions[0], dtype=np.float32)

i = 0
t0 = time.time()
for x, y in zip(X_test, Y_test):
    sequence = x[0]
    pH = x[1]
    length = x[2]
    
    row = np.array([amino_acid_data_dictionary[amino_acid] for amino_acid in sequence])
    row[:, [2,3, 4,5]] = row[:, [2,3, 4,5]] - pH
    row[:, 3] = row[:, 3] - (14-pH)
    row[:, 2:6] = 10 ** (row[:, 2:6])
    row[:, 2:6] = 1 / (row[:, 2:6] + 1)
    row = np.nan_to_num(row, nan=1.0)
    row = np.array([row[j:j+shingle_size].reshape(-1) for j in range(row.shape[0]-shingle_size)])
    
    X_test_shingled[i:i+(cutoff_length + 2*pad_length - shingle_size)] = row
    Y_test_shingled[i:i+(cutoff_length + 2*pad_length - shingle_size)] = y

    i += (cutoff_length + 2*pad_length - shingle_size)

    print(f"At {int(100 * i/shingled_dimensions[0])}%. Runtime: {int((time.time()-t0)/60)} minutes", end="\r")

dump_svmlight_file(X_test_shingled, Y_test_shingled, "test_values.svm.txt")
#X_test_shingled = pd.DataFrame(X_test_shingled, columns=column_names)
#Y_test_shingled = pd.DataFrame(Y_test_shingled.T, columns=["Tm"]).Tm
#X_test_shingled.to_csv("X_test.csv")
#Y_test_shingled.to_csv("Y_test.csv")



Index(['padded_sequence', 'pH', 'length'], dtype='object')
(27261, 3)
Index(['Letter', 'Molecular Weight', 'Residue Weight', 'pKa1', 'pKb2', 'pKx3',
       'pl4', 'H', 'VSC', 'P1', 'P2', 'SASA', 'NCISC', 'carbon', 'hydrogen',
       'nitrogen', 'oxygen', 'sulfur'],
      dtype='object')
1128


KeyboardInterrupt: 

In [2]:
dtrain = xgb.DMatrix('train_values.svm.txt?format=libsvm#cacheprefix')
dtest = xgb.DMatrix('X_test.csv?format=csv&label_column=0#cacheprefix')
Y_test = pd.read_csv("Y_test.csv", names=["num", "tm"], header=None)
Y_test = np.mean(Y_test.tm.values.reshape((1128, -1)), axis=0)


XGBoostError: [10:41:48] /home/conda/feedstock_root/build_artifacts/xgboost-split_1705649946806/work/dmlc-core/src/io/local_filesys.cc:86: LocalFileSystem.GetPathInfo: train_values.svm.txt error: No such file or directory
Stack trace:
  [bt] (0) /home/parashara/micromamba/envs/novozymes/lib/libxgboost.so(dmlc::LogMessageFatal::~LogMessageFatal()+0x6e) [0x7fe628f8c2ee]
  [bt] (1) /home/parashara/micromamba/envs/novozymes/lib/libxgboost.so(dmlc::io::LocalFileSystem::GetPathInfo(dmlc::io::URI const&)+0x328) [0x7fe62962d9e8]
  [bt] (2) /home/parashara/micromamba/envs/novozymes/lib/libxgboost.so(dmlc::io::InputSplitBase::InitInputFileInfo(std::__cxx11::basic_string<char, std::char_traits<char>, std::allocator<char> > const&, bool)+0x12e) [0x7fe62962ae7e]
  [bt] (3) /home/parashara/micromamba/envs/novozymes/lib/libxgboost.so(dmlc::io::InputSplitBase::Init(dmlc::io::FileSystem*, char const*, unsigned long, bool)+0x85) [0x7fe62962b485]
  [bt] (4) /home/parashara/micromamba/envs/novozymes/lib/l

In [4]:
print("depth\tweight\tMSE")
for depth in [2, 5, 8]:
    for weight in [1, 3, 6]:

        params = {
            "eval_metric" : "mae",
            "objective" : "reg:squarederror",
            "learning_rate" : 0.1,
            "max_depth" : depth,
            "min_child_weight" : weight,
            "gamma" : 0,
            "subsample" : 0.8,
            "colsample_bytree" : 0.8,
            "scale_pos_weight" : 1,
            "seed" : 2024,
        }
        
        xgb_model = xgb.train(params, dtrain, 
                              num_boost_round=100, 
                              verbose_eval=10)
        
        Y_pred = xgb_model.predict(dtest)
        Y_pred = np.mean(Y_pred.reshape((1128, -1)), axis=0)
        print(f"{depth}\t{weight}\t{np.mean(((-1/np.log(Y_pred)) - (-1/np.log(Y_test)))**2)}")

depth	weight	MSE


/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:12:41] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1705649946806/work/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


2	1	62.5598831193374


/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:16:16] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1705649946806/work/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


2	3	62.5598831193374


...

8	1	0.2755095029228308


/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [21:51:41] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1705649946806/work/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


8	3	4.975094802574843


/home/parashara/micromamba/envs/novozymes/lib/python3.11/site-packages/xgboost/core.py:160: UserWarning: [22:03:46] WARNING: /home/conda/feedstock_root/build_artifacts/xgboost-split_1705649946806/work/src/learner.cc:742: 
Parameters: { "n_estimators" } are not used.

  warnings.warn(smsg, UserWarning)


8	6	22.822033401959576


In [5]:
print("gamma\tMSE")
for g in [0, 0.1, 0.3, 0.5]:

    params = {
        "eval_metric" : "mae",
        "objective" : "reg:squarederror",
        "learning_rate" : 0.1,
        "max_depth" : 5,
        "min_child_weight" : 3,
        "gamma" : g,
        "subsample" : 0.8,
        "colsample_bytree" : 0.8,
        "scale_pos_weight" : 1,
        "seed" : 2024,
    }
    
    xgb_model = xgb.train(params, dtrain, 
                          num_boost_round=100, 
                          verbose_eval=10)
    
    Y_pred = xgb_model.predict(dtest)
    Y_pred = np.mean(Y_pred.reshape((1128, -1)), axis=0)
    print(f"{g}\t{np.mean(((-1/np.log(Y_pred)) - (-1/np.log(Y_test)))**2)}")

gamma	MSE
0	0.08120459241279403
0.1	14.209564477719363
0.3	10.437982436627522


In [8]:
print("subs\tcols\tMSE")
for ss in [0.6, 0.7, 0.8, 0.9]:
    for cs in [0.6, 0.7, 0.8, 0.9]:

        params = {
            "eval_metric" : "mae",
            "objective" : "reg:squarederror",
            "learning_rate" : 0.1,
            "max_depth" : 5,
            "min_child_weight" : 3,
            "gamma" : 0,
            "subsample" : ss,
            "colsample_bytree" : cs,
            "scale_pos_weight" : 1,
            "seed" : 2024,
        }
        
        xgb_model = xgb.train(params, dtrain, 
                              num_boost_round=100, 
                              verbose_eval=10)
        
        Y_pred = xgb_model.predict(dtest)
        Y_pred = np.mean(Y_pred.reshape((1128, -1)), axis=0)
        print(f"{ss}\t{cs}\t{np.mean(((-1/np.log(Y_pred)) - (-1/np.log(Y_test)))**2)}")

subs	cols	MSE
0.6	0.6	2.799080331694534
0.6	0.7	0.30779620541817626
0.6	0.8	2.1331222033367356
0.6	0.9	0.3534327598043672
0.7	0.6	8.114092692644652
0.7	0.7	11.99103680021679
0.7	0.8	0.2608935148117849
0.7	0.9	1.4686510644914603
0.8	0.6	2.546560428727751
0.8	0.7	9.149214278525237
0.8	0.8	0.08120459241279403
0.8	0.9	0.09703590424951294
0.9	0.6	0.23300579661072673
0.9	0.7	22.30234002735722
0.9	0.8	2.087487168199545
0.9	0.9	1.2569701306340646


In [1]:
import sys
from collections import defaultdict
import time

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
from sklearn.utils import shuffle
from sklearn.datasets import dump_svmlight_file
from sklearn.datasets import load_svmlight_file
import scipy
import xgboost as xgb

from utils.utils import *

print(f"Loading training and test data")

dtrain = xgb.DMatrix(f"train_all.svm.txt?format=libsvm#cacheprefix")
X_test = xgb.DMatrix('X_test.csv?format=csv#cacheprefix')

print(f"Training model")
params = {
    "eval_metric" : "mae",
    "objective" : "reg:squarederror",
    "learning_rate" : 0.1,
    "max_depth" : 5,
    "min_child_weight" : 3,
    "gamma" : 0,
    "subsample" : 0.8,
    "colsample_bytree" : 0.8,
    "scale_pos_weight" : 1,
    "seed" : 2024,
}

xgb_model = xgb.train(params, dtrain,
                      num_boost_round=100,
                      verbose_eval=10)




Loading training and test data
Training model


In [9]:
print("Prediction:")
X_test = np.loadtxt("X_test.csv", delimiter=",")
print(X_test.shape)
X_test = xgb.DMatrix(X_test)


Prediction:
(2721864, 85)


In [19]:

Y_pred = xgb_model.predict(X_test)

In [20]:
Y_pred = np.mean(Y_pred.reshape((1128, -1)), axis=0)
Y_pred = -1 / np.log(Y_pred)


In [21]:
Y_pred

array([60.11433 , 60.13426 , 60.115635, ..., 60.114765, 60.131847,
       60.123955], dtype=float32)

In [22]:
indices = list(range(31390, 31390 + len(Y_pred)))

In [23]:
Y_pred = pd.DataFrame(Y_pred, columns=['tm'], index=indices)

In [24]:
Y_pred.to_csv("Y_predictions_xgb.csv")